In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras.optimizers import Adam, SGD, Adadelta, Adagrad
from keras import Sequential, Model
from keras.models import load_model
from keras.utils import np_utils
from keras.metrics import Accuracy
from keras.preprocessing.image import ImageDataGenerator  
from keras.layers import (
    GlobalAveragePooling2D, Multiply, Flatten, Input,Lambda,
     Dense, Dropout, Conv2D, BatchNormalization, MaxPooling2D, Resizing)
from keras.callbacks import EarlyStopping
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras import activations
from vis.utils import utils

%pylab inline
import cv2
import glob
import os
import matplotlib.pylab as plt
import pickle
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import random
import seaborn as sns 
import shutil 
import time
import numpy as np

import pandas as pd

In [ ]:
#Funcion que genera los conjuntos de train y test de una carpeta concreta
def get_data(path, bt = 10):  
  train_generator = ImageDataGenerator(rotation_range=20,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       horizontal_flip=True,
                                       zoom_range = 0.2,
                                       shear_range = 0.2,
                                       vertical_flip = True ,
                                       preprocessing_function=preprocess_input)
  
  
  test_generator = ImageDataGenerator(preprocessing_function=preprocess_input)


  path_train = path + '/train'
  train_data = train_generator.flow_from_directory(directory= path_train,
                                                      class_mode='categorical',
                                                      shuffle = True, 
                                                      batch_size = bt,
                                                      seed=21,
                                                      classes=['Normal', 'Fracture'], 
                                                      target_size = (224,224)
                                                      )
  
  path_test = path + '/test'
  test_data = test_generator.flow_from_directory(directory= path_test,
                                                      class_mode='categorical',
                                                      shuffle = False,
                                                      batch_size = bt,
                                                      seed = 21,
                                                      classes=['Normal', 'Fracture'],
                                                      target_size = (224,224),
                                                      )
  
  return train_data, test_data

In [ ]:
# funcion que calcula el accuracy del modelo
def model_accuracy(model, data):
  y_pred = model.predict(data)
  y_pred = np.argmax(y_pred, axis=1)

  tn, fp, fn, tp =confusion_matrix(y_true =data.classes,y_pred =y_pred).ravel()
  return round((tn+tp)/(tn+fp+fn+tp),4)

In [ ]:
#devuelve el accuracy, el clasification report, la matriz de confusion y el ROC del modelo
def get_results(model, test_gen, acc = True, cm = True, cr = True, auc_ind = False, plot_roc = True):
  y_pred = model.predict(test_gen)
  y_predMax = np.argmax(y_pred, axis=1)

  if acc:
    acc_model = model_accuracy(model, test_gen)
    print('Accuracy: ', acc_model, '\n\n')

  if cr:
    print("Classification report")
    cr = classification_report(test_gen.classes, y_predMax, target_names=test_gen.class_indices, zero_division=0 )
    print(cr)
    print('\n\n')

  if cm:
    cm = confusion_matrix(y_true =test_gen.classes,
                          y_pred =y_predMax)

    ax= plt.subplot()
    sns.heatmap(cm, annot=True, fmt='g', ax=ax) 

    # labels, title and ticks
    lbls = labels = list(test_gen.class_indices.keys())
    ax.set_xlabel('Predicciones')
    ax.set_ylabel('Clases originales')
    ax.set_title('Matriz confusion')
    ax.xaxis.set_ticklabels(lbls) 
    plt.xticks(rotation=90)
    plt.yticks(np.arange(0.5, 2.5, 1.0))
    ax.yaxis.set_ticklabels(lbls)
    plt.yticks(rotation=0)

    plt.gcf().set_size_inches(6,4)
    plt.show()

    fpr, tpr, thresholds = roc_curve(test_gen.classes, y_predMax, pos_label=1)
    auc_value = round(auc(fpr, tpr),4)
    if auc_ind:
      print("AUC: ",auc_value )

    if plot_roc:
      plt.figure()
      lw = 2
      plt.plot(fpr, tpr,
          color="darkorange",
          lw=lw,
          label="curva ROC (area = %0.2f)" % auc_value,
      )
      plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
      plt.xlim([0.0, 1.0])
      plt.ylim([0.0, 1.05])
      plt.xlabel("Ratio Falsos Positivos")
      plt.ylabel("Ratio Verdadeos Positivos")
      plt.title("ROC")
      plt.legend(loc="lower right")
      plt.show()

In [ ]:
#Ejempo de avaluación del modelo
MODEL = load_model('/content/drive/MyDrive/TFM/PAC3_Modelos/Modelo_final/02_A_model/weights_model/SGD_0,1_40.pkl')
TRAIN_DATA, TEST_DATA = get_data('/content/drive/MyDrive/TFM/PAC3_Modelos/BBDD/Preprocessed Wrist Fracture A View', bt = 40)
get_results(MODEL,TEST_DATA, acc = False)

In [ ]:
#Creacion salency map

def smap(img_path, model_path, alpha = 0.4):
  #lectura de la imagen
  _img = keras.preprocessing.image.load_img(img_path,target_size=(224,224))

  #carga del modelo + modificar ultimo layer 
  model_saliency = load_model(model_path, compile = True)
  layer_idx = utils.find_layer_idx(model_saliency, 'predictions')
  model_saliency.layers[layer_idx].activation = activations.linear
  model_saliency = utils.apply_modifications(model_saliency)

  #aplicar transformaciones para calcular el gradiente 
  # siguiendo los pasos de 
  # https://usmanr149.github.io/urmlblog/cnn/2020/05/01/Salincy-Maps.html
  img = keras.preprocessing.image.img_to_array(_img)
  img = preprocess_input(img)
  img = img.reshape((1, *img.shape))
  images = tf.Variable(img, dtype=float)

  with tf.GradientTape() as tape:
      pred = model_saliency(images, training=False)
      class_idxs_sorted = np.argsort(pred.numpy().flatten())[::-1]
      loss = pred[0][class_idxs_sorted[0]]
      
  grads = tape.gradient(loss, images)
  dgrad_abs = tf.math.abs(grads)
  dgrad_max_ = np.max(dgrad_abs, axis=3)[0]
  arr_min, arr_max  = np.min(dgrad_max_), np.max(dgrad_max_)
  grad_eval = (dgrad_max_ - arr_min) / (arr_max - arr_min + 1e-18)


  # Sobreponer el saliencymap a la imagen original, paso a paso
  img = keras.preprocessing.image.img_to_array(_img)

  # Reescalar heatmap al rango 0-255
  heatmap = np.uint8(255 * grad_eval)

  # Uso de los valores RGB de los mapa de colores
  jet = cm.get_cmap("jet")
  jet_colors = jet(np.arange(256))[:, :3]
  jet_heatmap = jet_colors[heatmap]  
  
  # Crear la imagen a partir del heatmap colorizado RGB
  jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
  jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
  jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

  # Superponer el heatmap en la imagen original
  superimposed_img = jet_heatmap * alpha + img
  superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

  # Mostrar imagenes 
  fig, axes = plt.subplots(1,3,figsize=(30,12))
  axes[0].imshow(_img)
  axes[1].imshow(grad_eval,cmap="jet",alpha=1)
  axes[2].imshow(superimposed_img,cmap="jet", alpha=alpha)


In [ ]:
#ejemplo de salency map
IMG_PATH = os.path.join('/content/drive/MyDrive/TFM/PAC3_Modelos/BBDD/Preprocessed Wrist Fracture/test/Fracture/Resized 35 Male (A View).jpg')
MODEL_PATH = '/content/drive/MyDrive/TFM/PAC3_Modelos/Modelo_final/01_full_model/weights_model/SGD_0,1_40.pkl'
sal_map = smap(img_path=IMG_PATH,model_path = MODEL_PATH, alpha = 0.9)

In [ ]:
#Mostrar un error 
MODEL = load_model('/content/drive/MyDrive/TFM/PAC3_Modelos/Modelo_final/02_A_model/weights_model/SGD_0,1_40.pkl')
TRIAN_DATA, TEST_DATA = get_data('/content/drive/MyDrive/TFM/PAC3_Modelos/BBDD/Preprocessed Wrist Fracture', bt = 40)

y_pred = model.predict(TEST_DATA)
y_predMax = np.argmax(y_pred, axis=1)


list_ind = []
for i in range(0,len(y_predMax)):
  if y_predMax[i] != TEST_DATA.classes[i]:
    list_ind.append(i)


if len(list_ind) > 0:
  item = random.choice(list_ind) #elige solo 1 imagen de las mal etiquetadas
  path = TEST_DATA.filepaths[item] 
  # Identifiacion clase real y prediccion
  labels = list(TEST_DATA.class_indices.keys())
  clas_pred =labels[y_predMax[item]]
  clas_real =labels[TEST_DATA.classes[item]]
  print("Error: {},\nClasse predita: {},\nClasse real: {}".format(path,clas_pred,clas_real))

  # mostrar imagen
  random_img = plt.imread(path)
  plt.imshow(random_img)
  plt.figsize=(10,10)
  plt.show()